In [3]:
import requests
import json
import folium
import polyline

def geocode_address(address):
    """Convert an address to coordinates using Nominatim (OpenStreetMap's geocoder)"""
    # Use Nominatim API for geocoding (free)
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "json",
        "limit": 1
    }
    
    # Add a user-agent header (required by Nominatim's terms of use)
    headers = {
        "User-Agent": "air-quality-route-planner/1.0"
    }
    
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code != 200 or not response.json():
        print(f"Error: Could not geocode address '{address}'")
        return None
    
    # Extract latitude and longitude
    result = response.json()[0]
    lat = float(result["lat"])
    lon = float(result["lon"])
    display_name = result["display_name"]
    
    print(f"Geocoded '{address}' to '{display_name}' at coordinates: {lat}, {lon}")
    return (lat, lon)

def get_osrm_routes(start_address, end_address, alternatives=True):
    """Get routes between two addresses"""
    # First, geocode the addresses to coordinates
    start_coords = geocode_address(start_address)
    end_coords = geocode_address(end_address)
    
    if not start_coords or not end_coords:
        return None
    
    # OSRM expects coordinates as lon,lat (not lat,lon)
    start_lon, start_lat = start_coords[1], start_coords[0]
    end_lon, end_lat = end_coords[1], end_coords[0]
    
    # Construct the OSRM API URL
    url = f"https://router.project-osrm.org/route/v1/driving/{start_lon},{start_lat};{end_lon},{end_lat}"
    
    # Add parameters
    params = {
        "overview": "full",  # Get detailed route geometry
        "alternatives": "true" if alternatives else "false",  # Get alternative routes
        "steps": "true",     # Include turn-by-turn directions
        "annotations": "true" # Get additional data like speed
    }
    
    # Make the request
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    
    routes_data = response.json()
    
    # Return data and the resolved coordinates
    return {
        "routes_data": routes_data,
        "start_coords": start_coords,
        "end_coords": end_coords
    }

# Example usage
if __name__ == "__main__":
    # Use addresses instead of coordinates
    start_address = "Empire State Building, New York"
    end_address = "Fenway Park, Boston"
    
    print(f"Finding routes from '{start_address}' to '{end_address}'...")
    result = get_osrm_routes(start_address, end_address)
    
    if result:
        routes_data = result["routes_data"]
        start_coords = result["start_coords"]
        end_coords = result["end_coords"]
        
        print(f"Found {len(routes_data['routes'])} routes")
        
        # Print route summaries
        for i, route in enumerate(routes_data['routes']):
            distance_km = round(route['distance'] / 1000, 2)
            duration_min = round(route['duration'] / 60, 2)
            print(f"Route {i+1}: {distance_km} km, {duration_min} minutes")
        
        # Visualize routes on a map (using the function from previous example)
        map_viz = visualize_routes(routes_data, start_coords, end_coords)
        if map_viz:
            # Save map to HTML file
            map_viz.save('routes_map.html')
            print("\nMap saved to 'routes_map.html'")
    else:
        print("Failed to retrieve routes")

Finding routes from 'Empire State Building, New York' to 'Fenway Park, Boston'...
Geocoded 'Empire State Building, New York' to 'Empire State Building, 350, 5th Avenue, Koreatown, Manhattan Community Board 5, Manhattan, New York County, City of New York, New York, 10118, United States' at coordinates: 40.74844205, -73.98565890160751
Geocoded 'Fenway Park, Boston' to 'Fenway Park, 4, Jersey Street, Audubon Circle, Fenway-Kenmore, Boston, Suffolk County, Massachusetts, 02115, United States' at coordinates: 42.346461399999995, -71.09709985343964
Found 2 routes
Route 1: 336.81 km, 274.84 minutes
Route 2: 361.2 km, 295.94 minutes

Map saved to 'routes_map.html'
